# Data ETL pipeline

In [1]:
import sys
sys.path.append("../")
from src.service.provider import ProviderService
from src.prepare.data_etl import DataETL
import pandas as pd

provider = ProviderService()
# fix this
config = provider.categories.tdt_gemini

DATA_FOLDER = "tdt"
SIZE = 120
OVERLAP = 60

# provider.config.enable_tracing(project="ETL")
etl = DataETL(provider=provider, rag_config=config)

/home/h4438/miniconda3/envs/uni/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
a = etl.try_split_text(data_folder=DATA_FOLDER, size=SIZE, overlap=OVERLAP, use_sitemap=True)

From (59, 5) to Counter({18: 5, 15: 5, 16: 4, 24: 4, 20: 3, 19: 3, 13: 3, 8: 3, 30: 2, 28: 2, 44: 2, 14: 2, 25: 2, 17: 2, 4: 2, 12: 2, 6: 2, 2: 2, 35: 1, 23: 1, 33: 1, 34: 1, 21: 1, 40: 1, 11: 1, 41: 1, 9: 1}) -> 1116


## Reset 

In [2]:
etl.reset_config_repo(provider=provider, rag_config=config)

-> Delete tdt-vec-gemini
-> Delete tdt-text-split-gemini
DONE


## Upload docs

In [3]:
## OLD
# a = etl.upload_df_docs(
#     data_folder=DATA_FOLDER,
#     size=SIZE,
#     overlap=OVERLAP, re_update=False)

In [4]:
config.vector_index

'tdt-vec-gemini'

In [5]:
a = etl.upload_from_sitemap(
    data_folder=DATA_FOLDER, 
    size=SIZE, overlap=OVERLAP, re_update=True)

Total run: (59, 5)
-> DONE: if upload to elastic search 18 & 18 docs
-> DONE: upload to firebase at tdt
OK: 0
-> DONE: if upload to elastic search 30 & 30 docs
-> DONE: upload to firebase at tdt
OK: 1
-> DONE: if upload to elastic search 16 & 16 docs
-> DONE: upload to firebase at tdt
OK: 2
-> DONE: if upload to elastic search 15 & 15 docs
-> DONE: upload to firebase at tdt
OK: 3
-> DONE: if upload to elastic search 30 & 30 docs
-> DONE: upload to firebase at tdt
OK: 4
-> DONE: if upload to elastic search 35 & 35 docs
-> DONE: upload to firebase at tdt
OK: 5
-> DONE: if upload to elastic search 18 & 18 docs
-> DONE: upload to firebase at tdt
OK: 6
-> DONE: if upload to elastic search 24 & 24 docs
-> DONE: upload to firebase at tdt
OK: 7
-> DONE: if upload to elastic search 23 & 23 docs
-> DONE: upload to firebase at tdt
OK: 8
-> DONE: if upload to elastic search 18 & 18 docs
-> DONE: upload to firebase at tdt
OK: 9
-> DONE: if upload to elastic search 20 & 20 docs
-> DONE: upload to fi

In [5]:
print(a[-1].content)

A table of content
1. Ngành Bảo hộ lao động
2. Ngành Công nghệ sinh học
3. Ngành Công tác xã hội
4. Ngành Dược
5. Ngành Kế toán
6. Ngành Khoa học máy tính
7. Ngành Khoa học môi trường
8. Ngành Kiến trúc
9. Ngành Kinh doanh quốc tế
10. Ngành Kỹ thuật cơ điện tử
11. Ngành Kỹ thuật hóa học
12. Ngành Kỹ thuật phần mềm
13. Ngành Kỹ thuật Xây dựng
14. Ngành Kỹ thuật xây dựng công trình giao thông
15. Ngành Kỹ thuật điện
16. Ngành Kỹ thuật Điện tử - Viễn thông
17. Ngành Kỹ thuật điều khiển và tự động hóa
18. Ngành Luật
19. Ngành mạng máy tính và truyền thông dữ liệu
20. Ngành Marketing
21. Ngành ngôn ngữ Anh
22. Ngành ngôn ngữ Trung Quốc
23. Ngành quan hệ lao động
24. Ngành quản lý thể dục thể thao - Chuyên ngành Golf
25. Ngành quản lý thể dục thể thao - Chuyên ngành kinh doanh thể thao và tổ chức sự kiện
26. Ngành quản lý xây dựng
27. Ngành quản trị kinh doanh - Chuyên ngành Quản trị nguồn nhân lực
28. Ngành quản trị kinh doanh - Chuyên ngành Quản trị nhà hàng và khách sạn
29. Ngành Quy hoạc

In [4]:
len(a)
etl.index_page

'trường đại học Tôn Đức Thắng: Tiền thân của Trường...\ntrường đại học Tôn Đức Thắng: \nSỨ MẠNG của trường ...\n'

In [5]:
etl.delete_doc(a[0])

In [6]:
etl.delete_doc(a[1])

## Upload 1 doc

In [7]:
category = "test_qa"

etl.upload_doc(
    db_category=category,
    data=df.iloc[1],
    text_splitter=etl.create_splitter(), 
    es=provider.get_elasticsearch_store(index=category))

TypeError: upload_doc() got an unexpected keyword argument 'db_category'

In [ ]:
es = provider.get_elasticsearch_store(index=category)
a = es.similarity_search(query="bye")
a

[Document(page_content='Bye is a common phrase for departing', metadata={'id': '8af20725-b2e8-4f04-b5ce-0567de3a64ca'})]

In [ ]:
id = a[0].metadata['id']
a = etl.fire.read_document(collection=category, document_id=id)
a

In [ ]:
a.content

' Bye is a common phrase for departing'